In [2]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import tflearn
import pickle
import numpy as np
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression

In [3]:
# Load preprocessed datasets
with open('preprocess_x_2.pickle', 'rb') as handle:
    x_shuffled = pickle.load(handle)

with open('preprocess_y_2.pickle', 'rb') as handle:
    y_shuffled = pickle.load(handle)

print ("Files loaded.")
print ("x_shuffled size: {:d}".format(len(x_shuffled)))
print ("y_shuffled size: {:d}".format(len(y_shuffled)))

Files loaded.
x_shuffled size: 108
y_shuffled size: 108


In [7]:
# Split train/test set
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
for train_ind, test_ind in sss.split(x_shuffled, y_shuffled):
    print ("TRAIN:", train_ind, "TEST:", test_ind)
    X_train, X_val = x_shuffled[train_ind], x_shuffled[test_ind]
    y_train, y_val = y_shuffled[train_ind], y_shuffled[test_ind]

print("Train features dimensions: {:d}, {:d}".format(*X_train.shape))
print("Train labels dimensions: {:d}, {:d}".format(*y_train.shape))
print("Test features dimensions: {:d}, {:d}".format(*X_val.shape))
print("Test labels dimensions: {:d}, {:d}".format(*y_val.shape))

TRAIN: [ 47  67  10  73  15   0  83  80  11  91  36  34  18  12 100  37  35  48
  33  65  20  81  72  68  23  45 103  88  68  84  45  84  18  53   7  55
  98  74  95  30 103  62  96  96  28  64  80  67  82  10  15   7  31  92
  30  55  39  44  89  65   9  22  49  76  79  78   5  53   9  81  97  16
  86 107  69  70  42   4  69  99  25  70  79  38  60  26  40  56  17  63
  59  39  24 101  90  66  16] TEST: [43 27 66 73 19 90 19 57 54 29 88]
TRAIN: [ 74  17  26  19  87   2  52  55  74  10  30  34  77   6  51  48   0  86
  44  70  61  94  56  39  78  13  65  37   9  55  11  43 105  13   5  40
  83  69  45 101  89  14  99  58  70  85  35   4  81   7  50   1  67   2
  21  93  58  15   6  19  99  42 104  41  14  79  73  12   7  98 106  82
  71  95  37  24  80  27  10  20  36  93  90 100  49  40  72  92 102  84
  72  76  67  33  81  59  64] TEST: [23 76 49 71 94 25 44 69 83 12  3]
TRAIN: [ 54  54  86  37  31  26  61  57 105 105  85  48  20  87  99  90  83   8
  60  58  86  73 101  41  27  35  

In [14]:
'''# Building convolutional network
network = input_data(shape=[None, 407], name='input')
# Converts all words in vocabulary to lower dimensional representation
network = tflearn.embedding(network, input_dim=3800, output_dim=128)
branch1 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
branch2 = conv_1d(network, 128, 6, padding='valid', activation='relu', regularizer="L2")
branch3 = conv_1d(network, 128, 7, padding='valid', activation='relu', regularizer="L2")
network = merge([branch1, branch2, branch3], mode='concat', axis=1)
# Change the shape by adding 2 to dimensions
network = tf.expand_dims(network, 2)
network = global_max_pool(network)
network = dropout(network, 0.5)
network = fully_connected(network, 2, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')

# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(X_train, y_train, 
          n_epoch = 5, shuffle=True, 
          validation_set=(X_val, y_val), 
          show_metric=True, batch_size=32, 
          run_id='oc_1')'''

Training Step: 19  | total loss: 0.56784 | time: 1.757s
| Adam | epoch: 005 | loss: 0.56784 - acc: 0.8582 -- iter: 96/97
Training Step: 20  | total loss: 0.55129 | time: 2.790s
| Adam | epoch: 005 | loss: 0.55129 - acc: 0.9038 | val_loss: 0.56559 - val_acc: 0.8182 -- iter: 97/97
--


In [21]:
# Build TensorFlow model
sequence_length = X_train.shape[1]
num_classes = y_train.shape[1]
vocab_size = 15299
embedding_size = 128
filter_sizes = [5, 6, 7]
num_filters = 128
l2_reg_lambda = 0.0

graph = tf.Graph()
with graph.as_default():
    # Placeholders for input, output and dropout
    input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
    dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

    # Keep track of L2 regularization loss
    l2_loss = tf.constant(0.0)

    # Build model
    # Embedding layer
    with tf.device('/cpu:0'), tf.name_scope("embedding"):
        W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W")
        embedded_chars = tf.nn.embedding_lookup(W, input_x)
        embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)

    # Create a convolution + maxpool layer for each filter size
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            # Convolution layer
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W-%s" % filter_size)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b-%s" % filter_size)

            conv = tf.nn.conv2d(
                embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")

            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")

            # Maxpooling over the outputs
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, sequence_length - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="pool")
            pooled_outputs.append(pooled)

    # Combine all pooled features
    num_filters_total = num_filters * len(filter_sizes)
    h_pool = tf.concat(3, pooled_outputs)
    h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])

    # Add dropout
    with tf.name_scope("dropout"):
        h_drop = tf.nn.dropout(h_pool_flat, dropout_keep_prob)

    # Final (unnormalized) scores and predictions
    with tf.name_scope("output"):
        W = tf.get_variable(
            "W",
            shape=[num_filters_total, num_classes],
            initializer=tf.contrib.layers.xavier_initializer())

        b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
        l2_loss += tf.nn.l2_loss(W)
        l2_loss += tf.nn.l2_loss(b)
        scores = tf.nn.xw_plus_b(h_drop, W, b, name="scores")
        predictions = tf.argmax(scores, 1, name="predictions")

    # Calculate mean cross-entropy loss
    with tf.name_scope("loss"):
        losses = tf.nn.softmax_cross_entropy_with_logits(scores, input_y)
        loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

    # Accuracy
    with tf.name_scope("accuracy"):
        correct_predictions = tf.equal(predictions, tf.argmax(input_y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name="accuracy")
        
    # AUC
    with tf.name_scope("auc"):
        a = tf.cast(tf.argmax(predictions, 1),tf.float32)
        b = tf.cast(tf.argmax(input_y, 1),tf.float32)
        auc = tf.contrib.metrics.streaming_auc(a, b)

    # Optimizer
    global_step = tf.Variable(0, name="global_step", trainable=False)
    starter_learning_rate = 0.0005
    # Decay factor of 0.95 after every 10000 steps.
    with tf.name_scope('learning_rate'):
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 10000, 0.95)
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [22]:
# ======== Training =========
with tf.Session(graph=graph) as sess:
    #tf.initialize_all_variables().run()
    sess.run(tf.initialize_all_variables())
    sess.run(tf.initialize_local_variables())
    saver = tf.train.Saver(tf.all_variables())
    print('Initialized')

    def train_step(x_batch, y_batch):
        feed_dict = {
                input_x: x_batch,
                input_y: y_batch,
                dropout_keep_prob: 0.5
                }
        
        _, step, l, accuracy_train = sess.run(
            [optimizer, global_step, loss, accuracy], feed_dict=feed_dict)
        return step, l, accuracy_train
        
    def val_step(x_val, y_val):
        feed_dict = {
                input_x: x_val,
                input_y: y_val,
                dropout_keep_prob: 1.0
                }
            
        step, loss_val, accuracy_val = sess.run(
            [global_step, loss, accuracy], feed_dict=feed_dict)
        return accuracy_val
    
    def batch_iter(data, batch_size, num_epochs, shuffle=False):
        '''
        Generates a batch iterator for a dataset.
        '''
        data = np.array(data)
        data_size = len(data)
        num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
        for epoch in range(num_epochs):
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
            else:
                shuffled_data = data
            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                yield shuffled_data[start_index:end_index]

    batch_size = 64
    num_epochs = 50
    evaluate_every = 10
    checkpoint_every = 10
    checkpoint= '/home/ubuntu/pynb/oscars/cp'

    # Generate batches
    batches = batch_iter(
        list(zip(X_train, y_train)), batch_size, num_epochs)
    # Training loop. For each batch...
    for batch in batches:
        x_batch, y_batch = zip(*batch)
        step, l, accuracy_train = train_step(x_batch, y_batch)
        if (step % evaluate_every == 0):
            accuracy_val = val_step(X_val, y_val)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: {:.4f}'.format(accuracy_train))
            print('Validation accuracy: {:.4f}'.format(accuracy_val))
            #print('Minibatch AUC: {:.4f}'.format(auc_train))
            #print('Validation AUC: {:.4f}'.format(auc_val))
        if (step % checkpoint_every == 0):
            path = saver.save(sess, checkpoint, global_step=step)
            print("Saved model checkpoint to {}\n".format(path))

Initialized
Minibatch loss at step 10: 0.478298
Minibatch accuracy: 0.9091
Validation accuracy: 0.9091
Saved model checkpoint to /home/ubuntu/pynb/oscars/cp-10

Minibatch loss at step 20: 0.297160
Minibatch accuracy: 0.9697
Validation accuracy: 0.9091
Saved model checkpoint to /home/ubuntu/pynb/oscars/cp-20

Minibatch loss at step 30: 0.134719
Minibatch accuracy: 0.9697
Validation accuracy: 0.9091
Saved model checkpoint to /home/ubuntu/pynb/oscars/cp-30

Minibatch loss at step 40: 0.010866
Minibatch accuracy: 1.0000
Validation accuracy: 0.9091
Saved model checkpoint to /home/ubuntu/pynb/oscars/cp-40

Minibatch loss at step 50: 0.119697
Minibatch accuracy: 0.9697
Validation accuracy: 0.9091
Saved model checkpoint to /home/ubuntu/pynb/oscars/cp-50



KeyboardInterrupt: 

In [28]:
# Load preprocessed test dataset
'''with open('preprocess_test_1.pickle', 'rb') as handle:
    X_test = pickle.load(handle)
    
pred = model.predict(X_test)
print (pred)'''

[[0.37255746126174927, 0.6274425387382507], [0.37758776545524597, 0.6224122047424316], [0.3729100525379181, 0.6270899772644043], [0.37365567684173584, 0.6263443231582642], [0.38283562660217285, 0.6171643733978271]]
